In [1]:
#!/usr/bin/python
# -*- coding: UTF-8 -*-
import sys
import cv2
from PyQt5 import QtCore, QtGui, QtWidgets
from PyQt5.QtWidgets import *
from PyQt5.QtCore import *
from PyQt5.QtGui import QPalette, QBrush, QPixmap
import os

import sys

from statistics import mode

from keras.models import load_model
import numpy as np

from utils.datasets import get_labels

from utils.inference import detect_faces
from utils.inference import draw_text
from utils.inference import draw_bounding_box
from utils.inference import draw_solid_box
from utils.inference import apply_offsets
from utils.inference import load_detection_model
from utils.preprocessor import preprocess_input

from emotion_icon import load_emotion_icon
from emotion_icon import Addemotion
from emotion_icon import Addemotion_word

from DL_args import get_args
from keras.utils.data_utils import get_file
from contextlib import contextmanager
from wide_resnet import WideResNet
import argparse
from pathlib import Path
import dlib


import numpy as np
from utils.datasets import get_labels
from utils.inference import detect_faces

from tkinter import *
from PIL import Image,ImageTk
from utils.grad_cam import compile_gradient_function
from utils.grad_cam import compile_saliency_function
from utils.grad_cam import register_gradient
from utils.grad_cam import modify_backprop
from utils.grad_cam import calculate_guided_gradient_CAM
from utils.inference import detect_faces
from utils.inference import apply_offsets

from utils.datasets import get_class_to_arg

#################
from test_function import get_args
from test_function import draw_label
from test_function import video_capture
from test_function import yield_images
from test_function import yield_images_from_dir
#################


import Marquee
import tkinter as tk

global flag
flag = True

Using TensorFlow backend.


In [2]:
detection_model_path = '../trained_models/detection_models/haarcascade_frontalface_default.xml'
emotion_model_path = '../trained_models/emotion_models/fer2013_mini_XCEPTION.102-0.66.hdf5'
gender_model_path = '../trained_models/gender_models/simple_CNN.81-0.96.hdf5'
emotion_labels = get_labels('fer2013')
gender_labels = get_labels('imdb')
font = cv2.FONT_HERSHEY_SIMPLEX
face_detection = load_detection_model(detection_model_path)
emotion_classifier = load_model(emotion_model_path, compile=False)
gender_classifier = load_model(gender_model_path, compile=False)
gender_offsets = (30, 60)
emotion_offsets = (20, 40)
frame_window=10
# getting input model shapes for inference
emotion_target_size = emotion_classifier.input_shape[1:3]
gender_target_size = gender_classifier.input_shape[1:3]
icon_dict , words_dict = load_emotion_icon()

# starting lists for calculating modes


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [3]:
class Ui_MainWindow(QtWidgets.QWidget):
    def __init__(self, parent=None):
        super(Ui_MainWindow, self).__init__(parent)
        
        # self.face_recong = face.Recognition()
        self.timer_camera = QtCore.QTimer()
        self.cap = cv2.VideoCapture()
        self.CAM_NUM = 0
        self.set_ui()
        self.slot_init()
        self.__flag_work = 0
        self.x =0
        self.count = 0

    def set_ui(self):
        
        self.__layout_main = QtWidgets.QVBoxLayout() ##  垂直排版
        self.__layout_fun_button = QtWidgets.QHBoxLayout() ##  水平排版
        self.__layout_data_show = QtWidgets.QHBoxLayout()
        self.__layout_logo_show = QtWidgets.QHBoxLayout()
        
        ##
#         self.lb1 = QtWidgets.QLabel('實現夢想 在中正 ~！',self)
#         self.lb1.resize(300,500)
#         self.lb1.setFrameStyle(QFrame.Panel | QFrame.Sunken)
#         self.lb1.setAlignment(Qt.AlignBottom | Qt.AlignRight)
#         self.lb1.resultLabel.setText("<h2>實現夢想 在中正 ~！</h2>")
        ##
        # Set image on the button  start
        ICON_HEIGHT = 300 
        ICON_WIDTH = 200
        self.button_test = QtWidgets.QPushButton(u'')##  +button
        self.button_test.setIcon(QtGui.QIcon('./img/AR.png'))
        self.button_test.setIconSize(QtCore.QSize(ICON_HEIGHT,ICON_WIDTH))
        
        
        self.button_open_camera = QtWidgets.QPushButton(u'')
        self.button_open_camera.setIcon(QtGui.QIcon('./img/age_gender_emotion.png'))
        self.button_open_camera.setIconSize(QtCore.QSize(ICON_HEIGHT,ICON_WIDTH))
        
        
        self.button_close = QtWidgets.QPushButton(u'')
        self.button_close.setIcon(QtGui.QIcon('./img/face_fustion.PNG'))
        self.button_close.setIconSize(QtCore.QSize(ICON_HEIGHT,ICON_WIDTH))
        
        # Set image on the button  end
        
        #Button 的顏色修改
        button_color = [self.button_open_camera, self.button_close, self.button_test] ##  +button
        for i in range(3):
            button_color[i].setStyleSheet("QPushButton{color:black}"
                                          "QPushButton:hover{color:red}"
                                          "QPushButton{background-color:rgb(0,255,255)}"
                                          "QPushButton{border:2px}"
                                          "QPushButton{border-radius:10px}"
                                          "QPushButton{padding:2px 4px}")

        # set button size
        BUTTON_HEIGHT = 250
        BUTTON_WIDTH = 40
        self.button_open_camera.setMinimumHeight(BUTTON_HEIGHT)
        self.button_open_camera.setMinimumWidth(BUTTON_WIDTH)
        self.button_close.setMinimumHeight(BUTTON_HEIGHT)
        self.button_close.setMinimumWidth(BUTTON_WIDTH)
        self.button_test.setMinimumHeight(BUTTON_HEIGHT) # + button
        self.button_test.setMinimumWidth(BUTTON_WIDTH)
        # move()方法移動視窗在螢幕上的位置到x = 300，y = 300座標。
#         self.move(300,300)
        self.setGeometry(100, 100, 1217, 684)

    
        # 全大運圖片
        pix = QPixmap('./img/17.png')
        self.lb1 = QLabel()
        self.lb1.setFixedSize(300,300)
        self.lb1.setPixmap(pix)
        
        pix2 = QPixmap('./img/logo.png')
        self.lb2 = QLabel()
        self.lb2.setFixedSize(300,330)
        self.lb2.setPixmap(pix2)
        
        
        # 資訊顯示
        self.label_show_camera = QLabel()
        self.label_move = QtWidgets.QLabel()
        self.label_move.setFixedSize(80,100)  # Camera　frame　size

        self.label_show_camera.setFixedSize(1060, 1000)   # Main frame size
        self.label_show_camera.setAutoFillBackground(False)

#         self.__layout_fun_button.addWidget(self.label_move)

        # layer main
#         self.__layout_main.addLayout(self.__layout_data_show)
        self.__layout_main.addWidget(self.lb2)
        self.__layout_main.addWidget(self.label_show_camera)
        self.__layout_main.addLayout(self.__layout_data_show)
        self.__layout_main.addLayout(self.__layout_fun_button)
        
        # Layer data show
#         self.__layout_logo_show.addWidget(self.lb2)
#         self.__layout_logo_show.addWidget(self.lb2)
#         self.__layout_logo_show.addWidget(self.lb2)
        
        self.__layout_data_show.addWidget(self.lb1)
        self.__layout_data_show.addWidget(self.lb1)
        self.__layout_data_show.addWidget(self.lb1)
        
        # layer button
        self.__layout_fun_button.addWidget(self.button_open_camera)
        self.__layout_fun_button.addWidget(self.button_close)
        self.__layout_fun_button.addWidget(self.button_test) ## + button
        
        self.setLayout(self.__layout_main)
        self.label_move.raise_()
        self.setWindowTitle(u'攝像頭')

        
        # 設定背景圖片
        palette1 = QPalette()
        palette1.setBrush(self.backgroundRole(), QBrush(QPixmap('./img/background_3.jpg')))
        self.setPalette(palette1)

        

    def slot_init(self):
        self.button_open_camera.clicked.connect(self.button_open_camera_click)
        self.timer_camera.timeout.connect(self.show_camera)
        self.button_close.clicked.connect(self.close)
        self.button_test.clicked.connect(self.test_click)
    
    def test_click(self,flag):
        
#         face_fusion(0)
        
        if self.timer_camera.isActive() == False:
            flag = self.cap.open(self.CAM_NUM)
            if flag == False:
                msg = QtWidgets.QMessageBox.warning(self, u"Warning", u"請檢測相機與電腦是否連線正確", buttons=QtWidgets.QMessageBox.Ok,
                                                defaultButton=QtWidgets.QMessageBox.Ok)
            # if msg==QtGui.QMessageBox.Cancel:
            #                     pass
            else:
                self.timer_camera.start(60)
#                 self.button_open_camera.setText(u'關閉相機')
        else:
            self.timer_camera.stop()
            self.cap.release()
            self.label_show_camera.clear()
#             self.button_open_camera.setText(u'開啟相機')

        

    def button_open_camera_click(self):
        if self.timer_camera.isActive() == False:
            flag = self.cap.open(self.CAM_NUM)
            if flag == False:
                msg = QtWidgets.QMessageBox.warning(self, u"Warning", u"請檢測相機與電腦是否連線正確", buttons=QtWidgets.QMessageBox.Ok,
                                                defaultButton=QtWidgets.QMessageBox.Ok)
            # if msg==QtGui.QMessageBox.Cancel:
            #                     pass
            else:
                self.timer_camera.start(60)
                self.button_open_camera.setText(u'')
        else:
            self.timer_camera.stop()
            self.cap.release()
            self.label_show_camera.clear()
#             self.button_open_camera.setText(u'開啟相機')


    def show_camera(self):
        dlag, bgr_image = self.cap.read()
        if flag:
            gray_image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2GRAY)
            rgb_image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2RGB)
            faces = detect_faces(face_detection, gray_image)

            for face_coordinates in faces:
                x1, x2, y1, y2 = apply_offsets(face_coordinates, gender_offsets)
                rgb_face = rgb_image[y1:y2, x1:x2]

                x1, x2, y1, y2 = apply_offsets(face_coordinates, emotion_offsets)
                gray_face = gray_image[y1:y2, x1:x2]
                try:
                    rgb_face = cv2.resize(rgb_face, (gender_target_size))
                    gray_face = cv2.resize(gray_face, (emotion_target_size))
                except:
                    continue
                gray_face = preprocess_input(gray_face, False)
                gray_face = np.expand_dims(gray_face, 0)
                gray_face = np.expand_dims(gray_face, -1)
                emotion_label_arg = np.argmax(emotion_classifier.predict(gray_face))
                emotion_text = emotion_labels[emotion_label_arg]


                rgb_face = np.expand_dims(rgb_face, 0)
                rgb_face = preprocess_input(rgb_face, False)
                gender_prediction = gender_classifier.predict(rgb_face)
                gender_label_arg = np.argmax(gender_prediction)
                gender_text = gender_labels[gender_label_arg]
                
                set_icon = emotion_text+"_"+gender_text
        

                print(set_icon)
                icon_img = icon_dict[set_icon]
                words_img = words_dict[set_icon]


                
                if gender_text == gender_labels[0]:
                    color = (0, 0, 255)
                else:
                    color = (255, 0, 0)
        
#               if((face_coordinates[0:1]-face_coordinates[2:3])>icon_img[1]):
                print(face_coordinates[0])
                print(face_coordinates[2])
    
                draw_bounding_box(face_coordinates, rgb_image, color)
                solid_box = draw_solid_box(face_coordinates, rgb_image)
                solid_box = Addemotion(face_coordinates,solid_box,icon_img)
                solid_box = Addemotion_word(face_coordinates,solid_box,words_img)
#                 draw_text(face_coordinates, rgb_image, gender_mode,
#                         color, 0, -20, 1, 1)
#                 draw_text(face_coordinates, rgb_image, emotion_mode,
#                         color, 0, -45, 1, 1)
                
            show = cv2.resize(rgb_image, (1080, 960))
            showImage = QtGui.QImage(show, show.shape[1], show.shape[0], QtGui.QImage.Format_RGB888)
            self.label_show_camera.setPixmap(QtGui.QPixmap.fromImage(showImage))
    #bgr_image = cv2.cvtColor(rgb_image, cv2.COLOR_RGB2BGR)
    #cv2.imshow('window_frame', bgr_image)
    #if cv2.waitKey(1) & 0xFF == ord('q'):
        #exit(0)


        
    def closeEvent(self, event):
        ok = QtWidgets.QPushButton()
        cacel = QtWidgets.QPushButton()

        msg = QtWidgets.QMessageBox(QtWidgets.QMessageBox.Warning, u"關閉", u"是否關閉！")

        msg.addButton(ok,QtWidgets.QMessageBox.ActionRole)
        msg.addButton(cacel, QtWidgets.QMessageBox.RejectRole)
        ok.setText(u'確定')

        if msg.exec_() == QtWidgets.QMessageBox.RejectRole:
            event.ignore()
        else:
            #             self.socket_client.send_command(self.socket_client.current_user_command)
            if self.cap.isOpened():
                self.cap.release()
            if self.timer_camera.isActive():
                self.timer_camera.stop()
            event.accept() 
            



if __name__ == "__main__":
    App = QApplication(sys.argv)
    ex = Ui_MainWindow()
    ex.show()
    sys.exit(App.exec_())


happy_man
400
145
[400 263 145 145]
happy_man
354
136
[354 255 136 136]
happy_man
343
137
[343 255 137 137]
happy_man
336
143
[336 253 143 143]
happy_man
327
149
[327 248 149 149]
happy_man
334
140
[334 252 140 140]
happy_man
327
144
[327 252 144 144]
happy_man
328
139
[328 255 139 139]
happy_man
331
136
[331 257 136 136]
happy_man
332
135
[332 257 135 135]
happy_man
337
134
[337 258 134 134]
happy_man
332
147
[332 253 147 147]
happy_man
338
138
[338 255 138 138]
happy_man
342
137
[342 256 137 137]
happy_man
344
139
[344 255 139 139]
happy_man
356
137
[356 257 137 137]
happy_man
366
136
[366 257 136 136]
angry_man
372
147
[372 255 147 147]
disgust_man
381
141
[381 257 141 141]
happy_man
467
151
[467 275 151 151]
happy_man
463
155
[463 277 155 155]
happy_man
468
151
[468 281 151 151]
happy_man
472
151
[472 284 151 151]
happy_man
480
151
[480 287 151 151]
happy_woman
481
151
[481 285 151 151]


ValueError: operands could not be broadcast together with shapes (90,160) (90,159) 

happy_man
479
151
[479 298 151 151]
happy_man
478
151
[478 283 151 151]
happy_man
475
151
[475 280 151 151]
happy_man
469
157
[469 273 157 157]
happy_man
468
151
[468 273 151 151]
happy_man
464
154
[464 269 154 154]
happy_man
456
155
[456 267 155 155]
happy_man
461
150
[461 265 150 150]
happy_man
459
146
[459 265 146 146]
happy_man
448
154
[448 257 154 154]
happy_woman
452
142
[452 260 142 142]
happy_man
448
144
[448 258 144 144]
happy_man
438
158
[438 251 158 158]
happy_man
442
152
[442 255 152 152]
happy_woman
445
148
[445 254 148 148]
happy_man
443
149
[443 254 149 149]
happy_man
443
145
[443 253 145 145]
happy_man
445
144
[445 252 144 144]
happy_man
442
149
[442 252 149 149]
happy_man
444
144
[444 255 144 144]
happy_woman
444
144
[444 254 144 144]
happy_man
438
152
[438 251 152 152]
happy_man
440
149
[440 251 149 149]
happy_man
443
148
[443 253 148 148]
happy_man
441
149
[441 251 149 149]
happy_man
443
149
[443 251 149 149]
happy_man
440
153
[440 249 153 153]
happy_man
438
156
[438

ValueError: operands could not be broadcast together with shapes (90,160) (90,143) 

angry_woman
418
53
[418 212  53  53]
angry_woman
418
56
[418 212  56  56]
angry_woman
417
53
[417 212  53  53]
angry_man
416
53
[416 212  53  53]
angry_woman
418
53
[418 212  53  53]
angry_man
417
53
[417 212  53  53]
angry_woman
417
53
[417 212  53  53]
angry_woman
417
53
[417 212  53  53]
angry_woman
417
53
[417 212  53  53]
angry_woman
417
53
[417 212  53  53]
angry_woman
417
53
[417 212  53  53]
angry_woman
418
53
[418 212  53  53]
angry_woman
417
53
[417 212  53  53]
angry_woman
415
58
[415 210  58  58]
angry_woman
416
58
[416 210  58  58]
angry_woman
417
53
[417 212  53  53]
angry_woman
416
56
[416 212  56  56]
angry_woman
415
55
[415 211  55  55]
angry_woman
415
58
[415 210  58  58]
angry_woman
416
56
[416 211  56  56]
angry_woman
416
53
[416 212  53  53]
angry_woman
416
56
[416 211  56  56]
angry_woman
416
56
[416 211  56  56]
angry_woman
416
55
[416 211  55  55]
angry_woman
415
56
[415 211  56  56]
angry_woman
416
56
[416 211  56  56]
angry_woman
415
55
[415 211  55  55]
fear_

...

happy_man
369
62
[369  88  62  62]


ValueError: operands could not be broadcast together with shapes (90,160) (0,160) 

neutral_man
364
61
[364  92  61  61]
angry_man
360
62
[360  94  62  62]
angry_man
359
60
[359  99  60  60]
angry_man
358
58
[358 101  58  58]
angry_man
354
64
[354  98  64  64]
angry_man
357
58
[357 104  58  58]
angry_man
355
59
[355 104  59  59]
neutral_man
353
58
[353 106  58  58]
neutral_man
351
58
[351 107  58  58]
angry_man
348
61
[348 107  61  61]
neutral_man
344
63
[344 108  63  63]
angry_man
343
63
[343 108  63  63]
neutral_man
341
64
[341 109  64  64]
neutral_man
340
62
[340 110  62  62]
neutral_man
338
64
[338 111  64  64]
neutral_man
338
63
[338 112  63  63]
neutral_man
338
64
[338 111  64  64]
angry_man
335
64
[335 113  64  64]
angry_man
335
63
[335 114  63  63]
angry_man
335
62
[335 114  62  62]
neutral_man
334
64
[334 113  64  64]
happy_man
365
135
[365 317 135 135]
happy_man
352
147
[352 310 147 147]
happy_man
344
151
[344 309 151 151]
happy_man
347
135
[347 319 135 135]
neutral_man
335
62
[335 114  62  62]
angry_man
333
62
[333 114  62  62]
happy_man
336
151
[336 316 15

SystemExit: 0

C:\Users\robert\Anaconda3\envs\emotion_face\lib\site-packages\IPython\core\interactiveshell.py:3299: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
